In [1]:
#Import relevant Classes and make matplotlib inline to avoid loading plots at runtime
import descarteslabs as dl
import descarteslabs.workflows as wf
import config
import pandas as pd
import pickle as pkl
from gzip import compress, decompress
from descarteslabs import Storage, Auth

/opt/conda/lib/python3.7/site-packages/descarteslabs/_dl_modules/common/workflows/arrow_serialization/context.py:57: FutureWarning: 'pyarrow.SerializationContext' is deprecated and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  serialization_context = pa.SerializationContext()
/opt/conda/lib/python3.7/site-packages/descarteslabs/_dl_modules/common/workflows/arrow_serialization/context.py:58: FutureWarning: 'pyarrow.register_default_serialization_handlers' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  pa.register_default_serialization_handlers(serialization_context)


In [2]:
#Name of the current dataset, change as needed
datasetName='high_sampledworried_kitten'

In [3]:
#Initiate Storage
storage=Storage(auth=Auth(client_id=config.ID,client_secret=config.SECRET))
confirmed=pkl.loads(decompress(storage.get(datasetName+'Confirmed')))
denied=pkl.loads(decompress(storage.get(datasetName+'Denied')))

In [16]:
"""
totalDataset=pd.read_csv('../Predicted_Datasets/'+datasetName+'_results.csv').to_dict()
countryCount={}
countryMax={}
for x in totalDataset['area']:
    if totalDataset['area'][x] in countryMax.keys():
        countryMax[totalDataset['area'][x]]+=1
    else:
        countryMax[totalDataset['area'][x]]=1
    
for x in confirmed:
    idx=list(totalDataset['tile_key'].values()).index(str(x))
    country=totalDataset['area'][idx]
    if country in countryCount.keys():
        countryCount[country]+=1
    else:
        countryCount[country]=1

for x in countryCount.keys():
    print(x,countryCount[x],countryMax[x])
"""

"\ntotalDataset=pd.read_csv('../Predicted_Datasets/'+datasetName+'_results.csv').to_dict()\ncountryCount={}\ncountryMax={}\nfor x in totalDataset['area']:\n    if totalDataset['area'][x] in countryMax.keys():\n        countryMax[totalDataset['area'][x]]+=1\n    else:\n        countryMax[totalDataset['area'][x]]=1\n    \nfor x in confirmed:\n    idx=list(totalDataset['tile_key'].values()).index(str(x))\n    country=totalDataset['area'][idx]\n    if country in countryCount.keys():\n        countryCount[country]+=1\n    else:\n        countryCount[country]=1\n\nfor x in countryCount.keys():\n    print(x,countryCount[x],countryMax[x])\n"

In [17]:
"""
count=0
while len(storage.list(prefix=datasetName))>1 and count <5:
    print(datasetName)
    count+=1
"""

'\ncount=0\nwhile len(storage.list(prefix=datasetName))>1 and count <5:\n    print(datasetName)\n    count+=1\n'

In [18]:
#Make a unified group list with each key
totalGroups=[]
for groupBatch in list(storage.list(datasetName+'Group'))[:]:
    group=pkl.loads(decompress(storage.get(groupBatch)))
    for idx in group:
        totalGroups.extend(group[idx])

In [19]:
#Initiate the result dictionary
totalDataset=pd.read_csv('../Predicted_Datasets/'+datasetName+'_results.csv').to_dict()

In [20]:
#Create a predictions dict with key:prediction pairs
confirmedPred=[key for key in totalGroups if totalDataset['prediction'][list(totalDataset['tile_key'].values()).index(key)] >= .999]
deniedPred=[key for key in totalGroups if totalDataset['prediction'][list(totalDataset['tile_key'].values()).index(key)] < .999]
userConfirmed=[key for key in confirmed if key in totalGroups]
userDenied=[key for key in denied if key in totalGroups]

In [21]:
print(len(totalGroups))
print(len(confirmedPred)+len(deniedPred))
print(len(userDenied))
print(len(userConfirmed))
print(len(userDenied)+len(userConfirmed))

1100
1100
1048
52
1100


In [22]:
#Do Error Analysis!
fplist=[key for key in confirmedPred if key not in userConfirmed]
tplist=[key for key in confirmedPred if key in userConfirmed]
fnlist=[key for key in deniedPred if key not in userDenied]
tnlist=[key for key in deniedPred if key in userDenied]
fp=len(fplist)/(len(fplist)+len(tnlist))
tp=len(tplist)/(len(tplist)+len(fnlist))
fn=len(fnlist)/(len(fnlist)+len(tplist))
tn=len(tnlist)/(len(tnlist)+len(fplist))
p=len(tplist)/(len(tplist)+len(fplist))
r=len(tplist)/(len(tplist)+len(fnlist))
print('False Positive:',fp,'Len:',len(fplist))
print('True Positive:',tp,'Len:',len(tplist))
print('False Negative:',fn,'Len:',len(fnlist))
print('True Negative:',tn,'Len:',len(tnlist))
print('Precision:',p)
print('Recall:',r)

False Positive: 0.6526717557251909 Len: 684
True Positive: 0.7115384615384616 Len: 37
False Negative: 0.28846153846153844 Len: 15
True Negative: 0.3473282442748092 Len: 364
Precision: 0.05131761442441054
Recall: 0.7115384615384616
